In [1]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
from tqdm import tqdm
import os
import matplotlib.pyplot as plt

In [60]:
df1 = pd.read_csv(r'C:\Users\boyu571\boyu571_Github\01_Kakaobank_SKKU_Research_23\data\sentiment_score\sentiment_scoring_result1_prompt_ver1.csv', lineterminator='\n')
df2 = pd.read_csv(r'C:\Users\boyu571\boyu571_Github\01_Kakaobank_SKKU_Research_23\data\sentiment_score\result2_10300.csv', lineterminator='\n')
df3 = pd.read_csv(r'C:\Users\boyu571\boyu571_Github\01_Kakaobank_SKKU_Research_23\data\sentiment_score\result2_22208.csv', lineterminator='\n')
df4 = pd.read_csv(r'C:\Users\boyu571\boyu571_Github\01_Kakaobank_SKKU_Research_23\data\sentiment_score\sentiment_scoring_result3_prompt_ver1.csv', lineterminator='\n')

In [61]:
print(len(df1))
print(len(df2)+len(df3))
print(len(df4))

22209
22209
22209


In [62]:
# 각 DataFrame에서 첫 번째 열을 삭제
df1 = df1.drop(df1.columns[0], axis=1)
df2 = df2.drop(df2.columns[0], axis=1)
df3 = df3.drop(df3.columns[0], axis=1)
df4 = df4.drop(df4.columns[0], axis=1)

# 각 DataFrame에서 모든 열이 비어있는 행만 삭제
df1 = df1.dropna(how='all')
df3 = df3.dropna(how='all')
df3 = df3.dropna(how='all')
df4 = df4.dropna(how='all')

In [190]:
# import pandas as pd

# def combine_rows(df):
#     combined_rows = []  # 조합된 행을 저장할 리스트
#     temp_row = []  # 임시로 연속된 행들을 저장할 리스트
    
#     for index, row in df.iterrows():
#         # 비어있는 열의 수를 계산
#         empty_cols_count = row.isna().sum()
        
#         # 비어있는 열이 없는 경우
#         if empty_cols_count == 0:
#             if temp_row:  # temp_row가 비어있지 않은 경우
#                 combined = pd.concat(temp_row, ignore_index=True)
#                 # 결합된 행의 열의 개수가 df의 열의 개수보다 많은 경우
#                 while len(combined) > len(df.columns):
#                     # 4, 5 번째 열의 내용을 문자열 결합하여 하나의 열로 합침
#                     combined.iloc[3] = str(combined.iloc[3]) + " " + str(combined.iloc[4])
#                     combined = combined.drop(combined.index[4]).reset_index(drop=True)
#                 combined_rows.append(combined)
#                 temp_row.clear()
#             combined_rows.append(row)
#         else:  # 비어있는 열이 있는 경우
#             temp_row.append(row.dropna())
    
#     # 마지막 연속된 행들을 처리
#     if temp_row:
#         combined = pd.concat(temp_row, ignore_index=True)
#         # 결합된 행의 열의 개수가 df의 열의 개수보다 많은 경우
#         while len(combined) > len(df.columns):
#             # 4, 5 번째 열의 내용을 문자열 결합하여 하나의 열로 합침
#             combined.iloc[3] = str(combined.iloc[3]) + " " + str(combined.iloc[4])
#             combined = combined.drop(combined.index[4]).reset_index(drop=True)
#         combined_rows.append(combined)
    
#     # DataFrame으로 변환하고 원래의 컬럼 순서를 유지
#     combined_df = pd.DataFrame(combined_rows, columns=df.columns)
#     return combined_df

In [8]:
# def combine_rows(df):
#     combined_rows = []  # 조합된 행을 저장할 리스트
#     temp_row = []  # 임시로 연속된 행들을 저장할 리스트
#     for index, row in df.iterrows():
#         # 비어있는 열의 수를 계산
#         empty_cols_count = row.isna().sum()
        
#         # 비어있는 열이 없는 경우
#         if empty_cols_count == 0:
#             if temp_row:  # temp_row가 비어있지 않은 경우
#                 combined_rows.append(pd.concat(temp_row, ignore_index=True))
#                 temp_row.clear()
#             combined_rows.append(row)
#         else:  # 비어있는 열이 있는 경우
#             temp_row.append(row.dropna())
    
#     # 마지막 연속된 행들을 처리
#     if temp_row:
#         combined_rows.append(pd.concat(temp_row, ignore_index=True))
    
#     # DataFrame으로 변환하고 원래의 컬럼 순서를 유지
#     combined_df = pd.DataFrame(combined_rows, columns=df.columns)
#     return combined_df

In [163]:
# # 조합된 행들로 구성된 DataFrame을 얻음
# df2_combined = combine_rows(df2)
# df2_combined = df2_combined.dropna(how='all')
# df2_combined

,UTC-date,UTC-time,title,content,press,link,score_title
0,2016-12-31,2016-12-31 22:00:00,한국 금융산업 위기… ‘우버 모멘트’ 를 넘어라,© News1저금리·저성장 전통 금융산업 위기4차 산업혁명...세계는 핀테크 혁명 ...,뉴스1,https://n.news.naver.com/mnews/article/421/000...,0
1,2017-01-01,2017-01-01 06:15:04,"英 파운드, 2016년 최악의 통화 ···'디지털통화' 비트코인 최고의 수익자산...",【뉴욕=정지원 특파원】 영국의 파운드가 2106년 최악의 통화로 기록됐다. 파운드는...,파이낸셜뉴스,https://n.news.naver.com/mnews/article/014/000...,0.5
2,2017-01-01,2017-01-01 07:03:04,"[신년기획]블록체인, 금융권 넘어 IT기업도 관심",“블록체인은 인터넷 이후 나온 가장 혁신적이며 파괴적인 기술이다.”블록체인을 바라보...,39면 1단,https://n.news.naver.com/mnews/article/030/000...,0
3,2017-01-02,2017-01-02 09:47:22,공공기관·기업 대상으로 랜섬웨어 다량 유포…관계자 주의 필요,[아시아경제 박소연 기자]이스트소프트는 2일 국내 주요 국가 기관과 기업을 대상으로...,아시아경제,https://n.news.naver.com/mnews/article/277/000...,-1
4,2017-01-02,2017-01-02 08:04:01,블록체인 등 최신 핀테크 기술 도입… `금융 한류` 일으키자,올 세계 은행 80% '블록체인' 도입 전망금융회사 거래 비용 약 30% 절감 예측...,4면 TOP,https://n.news.naver.com/mnews/article/029/000...,0
...,...,...,...,...,...,...,...
10499,2021-01-02,2021-01-02 23:44:42,"""금·은 추가 상승여력 충분""",[파이낸셜뉴스] 2019년 7월 5일(현지시간) 스위스 노이카텔에서 촬영된 ...,파이낸셜뉴스언론사 선정,https://n.news.naver.com/mnews/article/014/000...,0
10500,2021-01-02,2021-01-02 14:53:57,"[속보] 2일 비트코인 3만달러 돌파 ""사상 최고가""",매일신문 속보 이미지. 매일신문DB2일(현지시간) 가상화폐(암호화폐_ 비트코인 가격...,매일신문언론사 선정,https://n.news.naver.com/mnews/article/088/000...,1
10501,2021-01-03,2021-01-03 07:18:17,[포토] 비트코인 가격 3천500만원 넘어서며 사상 최고가 기록,가상화폐 비트코인의 가격이 상승하며 2일(현지시간) 사상 최고가인 3만3천달러(약...,매일경제,https://n.news.naver.com/mnews/article/009/000...,1
10502,2021-01-03,2021-01-03 07:03:46,`디지털 엘도라도를 찾아서` 비트코인 3만달러 돌파…개미들 채굴업체 주식 매...,비트디지털·마라톤·라이엇 등투자자들 채굴업체 주식 매수올해 비트코인 ETF 첫 출시...,매일경제,https://n.news.naver.com/mnews/article/009/000...,0


In [38]:
# # 조합된 행들로 구성된 DataFrame을 얻음
# df1_combined = combine_rows(df1)
# df1_combined = df1_combined.dropna(how='all')

# # 조합된 행들로 구성된 DataFrame을 얻음
# df2_combined = combine_rows(df2)
# df2_combined = df2_combined.dropna(how='all')

# # 조합된 행들로 구성된 DataFrame을 얻음
# df3_combined = combine_rows(df3)
# df3_combined = df3_combined.dropna(how='all')

# # 조합된 행들로 구성된 DataFrame을 얻음
# df4_combined = combine_rows(df4)
# df4_combined = df4_combined.dropna(how='all')

In [198]:
# Find the indices of rows in df2_combined that have any missing values
rows_with_missing_values = df3_combined[df3_combined.isnull().any(axis=1)].index.tolist()
rows_with_missing_values

[]

In [64]:
df_full = df1
df_title = pd.concat([df2, df3])
df_content = df4
print(len(df_full))
print(len(df_title))
print(len(df_content))

22209
22209
22209


In [73]:
import pandas as pd
import re

def process_score(input_df):
    df = input_df.copy()  # 입력 DataFrame 복사하여 새로운 DataFrame 생성

    # 'score_title' 열을 float으로 변환하여 'score_title2' 열에 추가
    def convert_to_float(value):
        try:
            return float(value)
        except ValueError:
            return None

    df['score2'] = df['score'].apply(convert_to_float)

    # score_title2 열이 비어있는 행을 처리하여 값을 추출하여 채우기
    def fill_empty_score_title2(row):
        if pd.isna(row['score2']):
            # 문자열이 '\n'으로 이루어져 있는 경우 평균을 구함
            if '\n' in str(row['score']):
                if '컨퍼런스 개최 예정\n\n' in str(row['score']):
                    return float(0)
                else :
                    lines = row['score'].split("\n")
                    numbers = [float(line.split()[-1]) for line in lines]
                    average = sum(numbers) / len(numbers)
                    return average
            
            # 그렇지 않은 경우에는 원래대로 숫자를 추출
            else:
                numeric_values = re.findall(r'[-+]?\d*\.\d+|\d+', str(row['score']))
                if numeric_values:
                    numeric_value = float(numeric_values[0])
                    if -1 <= numeric_value <= 1:
                        return numeric_value
        return row['score2']

    df['score2'] = df.apply(fill_empty_score_title2, axis=1)
    
    return df

# 함수를 호출하여 결과 확인
processed_df_full = process_score(df_full)
processed_df_full

,UTC-date,UTC-time,title,content,press,link,score,score2
0,2016-12-31,2016-12-31 22:00:00,한국 금융산업 위기… ‘우버 모멘트’ 를 넘어라,© News1저금리·저성장 전통 금융산업 위기4차 산업혁명...세계는 핀테크 혁명 ...,뉴스1,https://n.news.naver.com/mnews/article/421/000...,0,0.0
1,2017-01-01,2017-01-01 06:15:04,"英 파운드, 2016년 최악의 통화 ···'디지털통화' 비트코인 최고의 수익자산...",【뉴욕=정지원 특파원】 영국의 파운드가 2106년 최악의 통화로 기록됐다. 파운드는...,파이낸셜뉴스,https://n.news.naver.com/mnews/article/014/000...,0,0.0
2,2017-01-01,2017-01-01 07:03:04,"[신년기획]블록체인, 금융권 넘어 IT기업도 관심",“블록체인은 인터넷 이후 나온 가장 혁신적이며 파괴적인 기술이다.”블록체인을 바라보...,39면 1단,https://n.news.naver.com/mnews/article/030/000...,0,0.0
3,2017-01-02,2017-01-02 09:47:22,공공기관·기업 대상으로 랜섬웨어 다량 유포…관계자 주의 필요,[아시아경제 박소연 기자]이스트소프트는 2일 국내 주요 국가 기관과 기업을 대상으로...,아시아경제,https://n.news.naver.com/mnews/article/277/000...,0,0.0
4,2017-01-02,2017-01-02 08:04:01,블록체인 등 최신 핀테크 기술 도입… `금융 한류` 일으키자,올 세계 은행 80% '블록체인' 도입 전망금융회사 거래 비용 약 30% 절감 예측...,4면 TOP,https://n.news.naver.com/mnews/article/029/000...,1,1.0
...,...,...,...,...,...,...,...,...
22204,2023-08-30,2023-08-30 02:13:01,"웨이브릿지, 빈터와 전통·가상자산 혼합형 전략 지수 발표",사진=웨이브릿지퀀트 기반 핀테크 기업 웨이브릿지가 글로벌 지수 빈터(Vinter)와...,한국경제,https://n.news.naver.com/mnews/article/015/000...,0,0.0
22205,2023-08-30,2023-08-30 02:14:01,"CJ ONE, 디지털 멤버십 NFT 발행",구매고객에 매달 최대 1만 포인트CJ올리브네트웍스가 운영하는 라이프스타일 멤버십 서...,16면 1단,https://n.news.naver.com/mnews/article/016/000...,0,0.0
22206,2023-08-30,2023-08-30 02:31:01,"‘신뢰’ 등에 업은 페이팔의 스테이블 코인, 시장의 메기가 될까[비트코인 A...",미국 캘리포니아주에 위치한 페이팔 본사./연합뉴스결제 공룡 페이팔이 스테이블 코인 ...,한경비즈니스,https://n.news.naver.com/mnews/article/050/000...,0,0.0
22207,2023-08-30,2023-08-30 01:50:07,"웨이브릿지, HAAM 전략 지수 개발…'코인 투자 손실률 최소화'",글로벌 지수 사업자 '빈터'와 맞손[ⓒ웨이브릿지][디지털데일리 박세아 기자] 퀀트 ...,디지털데일리,https://n.news.naver.com/mnews/article/138/000...,0,0.0


In [69]:
import pandas as pd
import re

def process_score_title(input_df):
    df = input_df.copy()  # 입력 DataFrame 복사하여 새로운 DataFrame 생성

    # 'score_title' 열을 float으로 변환하여 'score_title2' 열에 추가
    def convert_to_float(value):
        try:
            return float(value)
        except ValueError:
            return None

    df['score_title2'] = df['score_title'].apply(convert_to_float)

    # score_title2 열이 비어있는 행을 처리하여 값을 추출하여 채우기
    def fill_empty_score_title2(row):
        if pd.isna(row['score_title2']):
            # 문자열이 '\n'으로 이루어져 있는 경우 평균을 구함
            if '\n' in str(row['score_title']):
                if '컨퍼런스 개최 예정\n\n' in str(row['score_title']):
                    return float(0)
                lines = row['score_title'].split("\n")
                numbers = [float(line.split()[-1]) for line in lines]
                average = sum(numbers) / len(numbers)
                return average
            
            
            
            # 그렇지 않은 경우에는 원래대로 숫자를 추출
            else:
                numeric_values = re.findall(r'[-+]?\d*\.\d+|\d+', str(row['score_title']))
                if numeric_values:
                    numeric_value = float(numeric_values[0])
                    if -1 <= numeric_value <= 1:
                        return numeric_value
        return row['score_title2']

    df['score_title2'] = df.apply(fill_empty_score_title2, axis=1)
    
    return df

# 함수를 호출하여 결과 확인
processed_df_title = process_score_title(df_title)
processed_df_title

,UTC-date,UTC-time,title,content,press,link,score_title,score_title2
0,2016-12-31,2016-12-31 22:00:00,한국 금융산업 위기… ‘우버 모멘트’ 를 넘어라,© News1저금리·저성장 전통 금융산업 위기4차 산업혁명...세계는 핀테크 혁명 ...,뉴스1,https://n.news.naver.com/mnews/article/421/000...,0,0.0
1,2017-01-01,2017-01-01 06:15:04,"英 파운드, 2016년 최악의 통화 ···'디지털통화' 비트코인 최고의 수익자산...",【뉴욕=정지원 특파원】 영국의 파운드가 2106년 최악의 통화로 기록됐다. 파운드는...,파이낸셜뉴스,https://n.news.naver.com/mnews/article/014/000...,0.5,0.5
2,2017-01-01,2017-01-01 07:03:04,"[신년기획]블록체인, 금융권 넘어 IT기업도 관심",“블록체인은 인터넷 이후 나온 가장 혁신적이며 파괴적인 기술이다.”블록체인을 바라보...,39면 1단,https://n.news.naver.com/mnews/article/030/000...,0,0.0
3,2017-01-02,2017-01-02 09:47:22,공공기관·기업 대상으로 랜섬웨어 다량 유포…관계자 주의 필요,[아시아경제 박소연 기자]이스트소프트는 2일 국내 주요 국가 기관과 기업을 대상으로...,아시아경제,https://n.news.naver.com/mnews/article/277/000...,-1,-1.0
4,2017-01-02,2017-01-02 08:04:01,블록체인 등 최신 핀테크 기술 도입… `금융 한류` 일으키자,올 세계 은행 80% '블록체인' 도입 전망금융회사 거래 비용 약 30% 절감 예측...,4면 TOP,https://n.news.naver.com/mnews/article/029/000...,0,0.0
...,...,...,...,...,...,...,...,...
11904,2023-08-30,2023-08-30 02:13:01,"웨이브릿지, 빈터와 전통·가상자산 혼합형 전략 지수 발표",사진=웨이브릿지퀀트 기반 핀테크 기업 웨이브릿지가 글로벌 지수 빈터(Vinter)와...,한국경제,https://n.news.naver.com/mnews/article/015/000...,0.0,0.0
11905,2023-08-30,2023-08-30 02:14:01,"CJ ONE, 디지털 멤버십 NFT 발행",구매고객에 매달 최대 1만 포인트CJ올리브네트웍스가 운영하는 라이프스타일 멤버십 서...,16면 1단,https://n.news.naver.com/mnews/article/016/000...,0.0,0.0
11906,2023-08-30,2023-08-30 02:31:01,"‘신뢰’ 등에 업은 페이팔의 스테이블 코인, 시장의 메기가 될까[비트코인 A...",미국 캘리포니아주에 위치한 페이팔 본사./연합뉴스결제 공룡 페이팔이 스테이블 코인 ...,한경비즈니스,https://n.news.naver.com/mnews/article/050/000...,0.5,0.5
11907,2023-08-30,2023-08-30 01:50:07,"웨이브릿지, HAAM 전략 지수 개발…'코인 투자 손실률 최소화'",글로벌 지수 사업자 '빈터'와 맞손[ⓒ웨이브릿지][디지털데일리 박세아 기자] 퀀트 ...,디지털데일리,https://n.news.naver.com/mnews/article/138/000...,0.0,0.0


In [74]:
import pandas as pd
import re

def process_score_content(input_df):
    df = input_df.copy()  # 입력 DataFrame 복사하여 새로운 DataFrame 생성

    # 'score_title' 열을 float으로 변환하여 'score_title2' 열에 추가
    def convert_to_float(value):
        try:
            return float(value)
        except ValueError:
            return None

    df['score_content2'] = df['score_content'].apply(convert_to_float)

    # score_title2 열이 비어있는 행을 처리하여 값을 추출하여 채우기
    def fill_empty_score_title2(row):
        if pd.isna(row['score_content2']):
            # 문자열이 '\n'으로 이루어져 있는 경우 평균을 구함
            if '\n' in str(row['score_content']):
                if '컨퍼런스 개최 예정\n\n' in str(row['score_content']):
                    return float(0)
                lines = row['score_content'].split("\n")
                numbers = [float(line.split()[-1]) for line in lines]
                average = sum(numbers) / len(numbers)
                return average
            
            
            
            # 그렇지 않은 경우에는 원래대로 숫자를 추출
            else:
                numeric_values = re.findall(r'[-+]?\d*\.\d+|\d+', str(row['score_content']))
                if numeric_values:
                    numeric_value = float(numeric_values[0])
                    if -1 <= numeric_value <= 1:
                        return numeric_value
        return row['score_content2']

    df['score_content2'] = df.apply(fill_empty_score_title2, axis=1)
    
    return df

# 함수를 호출하여 결과 확인
processed_df_content = process_score_content(df_content)
processed_df_content

,UTC-date,UTC-time,title,content,press,link,score_content,score_content2
0,2016-12-31,2016-12-31 22:00:00,한국 금융산업 위기… ‘우버 모멘트’ 를 넘어라,© News1저금리·저성장 전통 금융산업 위기4차 산업혁명...세계는 핀테크 혁명 ...,뉴스1,https://n.news.naver.com/mnews/article/421/000...,0,0.0
1,2017-01-01,2017-01-01 06:15:04,"英 파운드, 2016년 최악의 통화 ···'디지털통화' 비트코인 최고의 수익자산...",【뉴욕=정지원 특파원】 영국의 파운드가 2106년 최악의 통화로 기록됐다. 파운드는...,파이낸셜뉴스,https://n.news.naver.com/mnews/article/014/000...,0,0.0
2,2017-01-01,2017-01-01 07:03:04,"[신년기획]블록체인, 금융권 넘어 IT기업도 관심",“블록체인은 인터넷 이후 나온 가장 혁신적이며 파괴적인 기술이다.”블록체인을 바라보...,39면 1단,https://n.news.naver.com/mnews/article/030/000...,0,0.0
3,2017-01-02,2017-01-02 09:47:22,공공기관·기업 대상으로 랜섬웨어 다량 유포…관계자 주의 필요,[아시아경제 박소연 기자]이스트소프트는 2일 국내 주요 국가 기관과 기업을 대상으로...,아시아경제,https://n.news.naver.com/mnews/article/277/000...,0,0.0
4,2017-01-02,2017-01-02 08:04:01,블록체인 등 최신 핀테크 기술 도입… `금융 한류` 일으키자,올 세계 은행 80% '블록체인' 도입 전망금융회사 거래 비용 약 30% 절감 예측...,4면 TOP,https://n.news.naver.com/mnews/article/029/000...,0,0.0
...,...,...,...,...,...,...,...,...
22204,2023-08-30,2023-08-30 02:13:01,"웨이브릿지, 빈터와 전통·가상자산 혼합형 전략 지수 발표",사진=웨이브릿지퀀트 기반 핀테크 기업 웨이브릿지가 글로벌 지수 빈터(Vinter)와...,한국경제,https://n.news.naver.com/mnews/article/015/000...,0,0.0
22205,2023-08-30,2023-08-30 02:14:01,"CJ ONE, 디지털 멤버십 NFT 발행",구매고객에 매달 최대 1만 포인트CJ올리브네트웍스가 운영하는 라이프스타일 멤버십 서...,16면 1단,https://n.news.naver.com/mnews/article/016/000...,0,0.0
22206,2023-08-30,2023-08-30 02:31:01,"‘신뢰’ 등에 업은 페이팔의 스테이블 코인, 시장의 메기가 될까[비트코인 A...",미국 캘리포니아주에 위치한 페이팔 본사./연합뉴스결제 공룡 페이팔이 스테이블 코인 ...,한경비즈니스,https://n.news.naver.com/mnews/article/050/000...,0.5,0.5
22207,2023-08-30,2023-08-30 01:50:07,"웨이브릿지, HAAM 전략 지수 개발…'코인 투자 손실률 최소화'",글로벌 지수 사업자 '빈터'와 맞손[ⓒ웨이브릿지][디지털데일리 박세아 기자] 퀀트 ...,디지털데일리,https://n.news.naver.com/mnews/article/138/000...,0,0.0


In [75]:
processed_df_content.to_csv('score_content.csv', encoding='utf-8-sig', index=False)
processed_df_full.to_csv('score_full.csv', encoding='utf-8-sig', index=False)
processed_df_title.to_csv('score_title.csv', encoding='utf-8-sig', index=False)